#### 0. Manually download the file and insert into a Cloud Storage bucket

Results: The test was successful.

#### 1. Try to insert directly the file into BigQuery as a table

Results: I used the "infer schema" configuration, it was not successfull, maybe I have to use in python

#### 2. Define only the schema of the columns to be used in BigQuery (Manually)
I used a scrypt like this:

```
import json

data = []

# Reading data from the input file
input_file = 'farmers-protest-tweets-2021-2-4.json'
output_file = 'processed_tweets.jsonl'

with open(input_file, 'r') as f:
    for line in f:
        obj = json.loads(line)
        id = obj["id"]
        username = obj["user"]["username"]
        userid = obj["user"]["id"]
        date = obj["date"]
        content = obj["content"]

        list_mentioned_users = []

        if obj["mentionedUsers"] !=None:
            for j in obj["mentionedUsers"]:
                i = j
                mentioned_user = {
                    "username": i["username"],
                    "userid": i["id"]
                }
                list_mentioned_users.append(mentioned_user)
        else:
            pass
        data.append({"id": id, "username": username, "userid": userid, "date": date, "content": content, "mentionedUsers": list_mentioned_users})

filtered_data = [tweet for tweet in data]

# Writing the processed data to a new JSONL file
with open(output_file, 'w') as outfile:
    for tweet in filtered_data:
        json.dump(tweet, outfile)
        outfile.write('\n')  # Add a newline character to separate each tweet

# Optionally, you can print a message once the writing is complete
print(f"Processed data has been written to '{output_file}'")
```

The result was successfull.

#### 3. Create the queries in BigQuery
The test was sucessfull

I create the queries needed to retrieve the results

In [ ]:
# Question1
with highdates as (
SELECT date(`date`) `date`, count(1)
FROM `de-leonardo-burbano.DE_BIGQUERY_LB.test0006`
group by date(`date`)
order by 2 desc
limit 10)
, users as (
select b.username, date(b.`date`) as date_agg, COUNT(1) as num_tweets
from `de-leonardo-burbano.DE_BIGQUERY_LB.test0006` as b
inner join highdates on highdates.`date` = date(B.`date`)
group by b.username, date(b.`date`)
)
, c as (
select username, date_agg, row_number() over(partition by date_agg order by num_tweets desc) as num
from users)
select *
from c
where num = 1

: 

In [ ]:
#Question2
WITH A AS (
  select REGEXP_EXTRACT_ALL(content, r"(?:[\x{1F300}-\x{1F5FF}]|[\x{1F900}-\x{1F9FF}]|[\x{1F600}-\x{1F64F}]|[\x{1F680}-\x{1F6FF}]|[\x{2600}-\x{26FF}]\x{FE0F}?|[\x{2700}-\x{27BF}]\x{FE0F}?|\x{24C2}\x{FE0F}?|[\x{1F1E6}-\x{1F1FF}]{1,2}|[\x{1F170}\x{1F171}\x{1F17E}\x{1F17F}\x{1F18E}\x{1F191}-\x{1F19A}]\x{FE0F}?|[\\x{0023}\x{002A}\x{0030}-\x{0039}]\x{FE0F}?\x{20E3}|[\x{2194}-\x{2199}\x{21A9}-\x{21AA}]\x{FE0F}?|[\x{2B05}-\x{2B07}\x{2B1B}\x{2B1C}\x{2B50}\x{2B55}]\x{FE0F}?|[\x{2934}\x{2935}]\x{FE0F}?|[\x{3297}\x{3299}]\x{FE0F}?|[\x{1F201}\x{1F202}\x{1F21A}\x{1F22F}\x{1F232}\x{1F23A}\x{1F250}\x{1F251}]\x{FE0F}?|[\x{203C}-\x{2049}]\x{FE0F}?|[\x{00A9}-\x{00AE}]\x{FE0F}?|[\x{2122}\x{2139}]\x{FE0F}?|\x{1F004}\x{FE0F}?|\x{1F0CF}\x{FE0F}?|[\x{231A}\x{231B}\x{2328}\x{23CF}\x{23E9}\x{23F3}\x{23F8}\x{23FA}]\x{FE0F}?)") AS emoji
  from `de-leonardo-burbano.DE_BIGQUERY_LB.test0006`
  WHERE ARRAY_LENGTH(REGEXP_EXTRACT_ALL(content, r"(?:[\x{1F300}-\x{1F5FF}]|[\x{1F900}-\x{1F9FF}]|[\x{1F600}-\x{1F64F}]|[\x{1F680}-\x{1F6FF}]|[\x{2600}-\x{26FF}]\x{FE0F}?|[\x{2700}-\x{27BF}]\x{FE0F}?|\x{24C2}\x{FE0F}?|[\x{1F1E6}-\x{1F1FF}]{1,2}|[\x{1F170}\x{1F171}\x{1F17E}\x{1F17F}\x{1F18E}\x{1F191}-\x{1F19A}]\x{FE0F}?|[\\x{0023}\x{002A}\x{0030}-\x{0039}]\x{FE0F}?\x{20E3}|[\x{2194}-\x{2199}\x{21A9}-\x{21AA}]\x{FE0F}?|[\x{2B05}-\x{2B07}\x{2B1B}\x{2B1C}\x{2B50}\x{2B55}]\x{FE0F}?|[\x{2934}\x{2935}]\x{FE0F}?|[\x{3297}\x{3299}]\x{FE0F}?|[\x{1F201}\x{1F202}\x{1F21A}\x{1F22F}\x{1F232}\x{1F23A}\x{1F250}\x{1F251}]\x{FE0F}?|[\x{203C}-\x{2049}]\x{FE0F}?|[\x{00A9}-\x{00AE}]\x{FE0F}?|[\x{2122}\x{2139}]\x{FE0F}?|\x{1F004}\x{FE0F}?|\x{1F0CF}\x{FE0F}?|[\x{231A}\x{231B}\x{2328}\x{23CF}\x{23E9}\x{23F3}\x{23F8}\x{23FA}]\x{FE0F}?)"))>0
)
select element, count(1) as frecuency
FROM A, UNNEST(emoji) AS element
GROUP BY element
order by 2 desc
limit 10

In [ ]:
#Question3
with a as (
  SELECT mentioned.username mentioned_username, mentioned.userid mentioned_userid, A.id, A.username
  FROM  `de-leonardo-burbano.DE_BIGQUERY_LB.test0006` AS A, UNNEST(A.mentionedUsers) as mentioned
  WHERE mentioned.username is not null
)
select mentioned_username, count(1)
from a
where a.mentioned_username != username
group by 1
order by 2 desc
limit 10

#### Next steps
1. Configure the repo: requirements + gitignore
2. Define a strategy in the main Notebook